DATA 643 Project 1:

Angus, Pevan, Nathan

We built a simple recommender system for American TV sit-coms. This system will take user input for a sit com and predict the user's preference for possible unwatched sitcoms. This is similar in approach to systems used by Netflix, Hulu and Amazon Prime.

We used Facebook to collect data on 5 popular American TV sitcoms: The Big Bang Theory,	Brooklyn 99, It's Always Sunny in Philadelphia,	The Goldbergs, and The Unbreakable Kimmy Schmidt. Facebook friends were asked to rate these shows from 1 for 'Awful' to 5 for 'Outstanding' and to leave blank un-watched shows using a Google Docs Quiz. The data from the Google We will in turn use these data to generate a basic recommendation system for a test set of withheld data and unwatched shows. 

• Load your data into (for example) an R or pandas dataframe, a Python dictionary or list of lists, (or another data structure of your choosing). From there, create a user-item matrix.
• If you choose to work with a large dataset, you’re encouraged to also create a small, relatively dense “user-item” matrix as a subset so that you can hand-verify your calculations.
• Break your ratings into separate training and test datasets.
• Using your training data, calculate the raw average (mean) rating for every user-item combination.
• Calculate the RMSE for raw average for both your training data and your test data.
• Using your training data, calculate the bias for each user and each item.
• From the raw average, and the appropriate user and item biases, calculate the baseline predictors for every user-item combination.
• Calculate the RMSE for the baseline predictors for both your training data and your test data.
• Summarize your results.

In [200]:
#import desired packages here:
import pandas as pd 

In [201]:
tv_df = pd.read_csv('https://raw.githubusercontent.com/ncooper76/MSDA/master/TV_Show_Preferences.csv')
tv_df = tv_df.drop(columns=["Timestamp"])
tv_df = tv_df[sorted(tv_df.columns)]
tv_df

,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,5.0,NaN,1.0,2.0,3.0
1,2.0,3.0,3.0,2.0,1.0
2,NaN,NaN,4.0,NaN,NaN
3,4.0,NaN,1.0,3.0,5.0
4,NaN,2.0,2.0,NaN,3.0
5,3.0,NaN,1.0,3.0,4.0
6,4.0,1.0,3.0,1.0,2.0
7,NaN,NaN,5.0,NaN,5.0
8,NaN,1.0,3.0,1.0,NaN
9,5.0,NaN,3.0,NaN,NaN


In [202]:
tv_df_mean = tv_df.stack().mean(skipna = True) #stack makes the DF one column, mean takes the mean of that colunm.
print(tv_df_mean) # 3.1279... agrees with the average taken in Google Sheets
#below are the means for the rows and the columns of the dataframe
tv_df_col_means = tv_df.mean(axis=0,skipna = True)
print(tv_df_col_means)
tv_df_row_means = tv_df.mean(axis=1,skipna = True)
print(tv_df_row_means)

3.127906976744186
Brooklyn 99                          3.500000
It's Always Sunny in Philadelphia    2.615385
The Big Bang Theory                  3.347826
The Goldbergs                        2.800000
Unbreakable Kimmy Schmidt            3.157895
dtype: float64
0     2.750000
1     2.200000
2     4.000000
3     3.250000
4     2.333333
5     2.750000
6     2.200000
7     5.000000
8     1.666667
9     4.000000
10    4.000000
11    3.500000
12    3.750000
13    5.000000
14    3.600000
15    4.500000
16    2.400000
17    4.400000
18    3.800000
19    5.000000
20    2.000000
21    3.666667
22    3.000000
23    1.600000
dtype: float64


Now we will break into training and test sets and calculate the raw averages:

In [203]:
from sklearn.model_selection import train_test_split
tv_col_df = tv_df.stack()
tv_col_df.head(20)

tv_train, tv_test = train_test_split(tv_col_df, test_size =0.2)
tv_train = tv_train.sort_index()
tv_test = tv_test.sort_index()
#print(tv_train.head())
#print(tv_test.head())
train_raw_ave = tv_train.mean()
test_raw_ave = tv_test.mean()
print("The Raw Average on the Training Set is: " + str(train_raw_ave) + "\n")
print("The Raw Average on the Test Set is: " + str(test_raw_ave) + "\n")

The Raw Average on the Training Set is: 3.0

The Raw Average on the Test Set is: 3.611111111111111



Now we recast the data frames into their original wide format and calculate the row and column averages to calculate the biases:

In [204]:
tv_train_wide = tv_train.unstack().sort_index()
tv_train_col_means = tv_train_wide.mean(axis=0,skipna = True)
print(str(tv_train_col_means))
tv_train_row_means = tv_train_wide.mean(axis=1,skipna = True)
print(str(tv_train_row_means))

Brooklyn 99                          3.166667
It's Always Sunny in Philadelphia    2.500000
The Big Bang Theory                  3.300000
The Goldbergs                        2.727273
Unbreakable Kimmy Schmidt            3.076923
dtype: float64
0     2.750000
1     2.200000
2     4.000000
3     3.000000
4     2.333333
5     2.666667
6     2.250000
7     5.000000
8     1.666667
9     4.000000
10    4.000000
11    3.000000
12    3.500000
13    5.000000
14    3.750000
15    4.666667
16    2.400000
17    4.333333
18    3.333333
19    5.000000
20    2.000000
21    3.666667
22    3.000000
23    1.750000
dtype: float64


In [205]:
tv_test_wide = tv_test.unstack().sort_index()
tv_test_wide
tv_test_col_means = tv_test_wide.mean(axis=0,skipna = True)
print(str(tv_test_col_means))
tv_test_row_means = tv_test_wide.mean(axis=1,skipna = True)
print(str(tv_test_row_means))

Brooklyn 99                          4.500000
It's Always Sunny in Philadelphia    4.000000
The Big Bang Theory                  3.666667
The Goldbergs                        3.000000
Unbreakable Kimmy Schmidt            3.333333
dtype: float64
3     3.333333
5     3.000000
6     2.000000
10    4.000000
11    5.000000
12    4.000000
14    3.000000
15    4.000000
17    4.500000
18    4.500000
22    3.000000
23    1.000000
dtype: float64


Now we can calculate the RMSE for the raw averages:

In [206]:
from sklearn.metrics import mean_squared_error

train_pred = [tv_train.mean()]*len(tv_train)# we need a list of raw averages the same length as the stacked df
train_pred
train_rmse = mean_squared_error(tv_train, train_pred)**0.5
print("RMSE on the Training Set is: " + str(train_rmse) + "\n")
test_pred = [tv_train.mean()]*len(tv_test) # we need a list of raw averages the same length as the stacked df
test_pred
test_rmse = mean_squared_error(tv_test, test_pred)**0.5
print("RMSE on the Test Set is: " + str(test_rmse) + "\n")

RMSE on the Training Set is: 1.4245742398014511

RMSE on the Test Set is: 1.4337208778404378



Now we will use the row (user) and colunm (item) means to calculate bias, and use those biases to make better predictions for every user-item pair.

In [207]:
show_bias = tv_train_col_means - train_raw_ave 
print(show_bias)
user_bias = tv_train_row_means - train_raw_ave 
print(user_bias) 
tv_pred = pd.DataFrame(index=tv_df.index, columns=tv_df.columns.values)
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
            tv_pred.iloc[i,j] = train_raw_ave + user_bias[i] + show_bias[j]
            if tv_pred.iloc[i,j] < 1.0:
                tv_pred.iloc[i,j] = 1.0
            elif tv_pred.iloc[i,j] > 5.0:
                tv_pred.iloc[i,j] = 5.0
#tv_pred

Brooklyn 99                          0.166667
It's Always Sunny in Philadelphia   -0.500000
The Big Bang Theory                  0.300000
The Goldbergs                       -0.272727
Unbreakable Kimmy Schmidt            0.076923
dtype: float64
0    -0.250000
1    -0.800000
2     1.000000
3     0.000000
4    -0.666667
5    -0.333333
6    -0.750000
7     2.000000
8    -1.333333
9     1.000000
10    1.000000
11    0.000000
12    0.500000
13    2.000000
14    0.750000
15    1.666667
16   -0.600000
17    1.333333
18    0.333333
19    2.000000
20   -1.000000
21    0.666667
22    0.000000
23   -1.250000
dtype: float64


,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,2.91667,2.25,3.05,2.47727,2.82692
1,2.36667,1.7,2.5,1.92727,2.27692
2,4.16667,3.5,4.3,3.72727,4.07692
3,3.16667,2.5,3.3,2.72727,3.07692
4,2.5,1.83333,2.63333,2.06061,2.41026
5,2.83333,2.16667,2.96667,2.39394,2.74359
6,2.41667,1.75,2.55,1.97727,2.32692
7,5,4.5,5,4.72727,5
8,1.83333,1.16667,1.96667,1.39394,1.74359
9,4.16667,3.5,4.3,3.72727,4.07692


In [208]:
import math
tv_pred_train = pd.DataFrame(index=tv_df.index, columns=tv_df.columns.values)
tv_pred_train = tv_pred_train[sorted(tv_pred_train.columns)]
tv_train_wide = tv_train_wide[sorted(tv_train_wide.columns)]
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
        if math.isnan(tv_train_wide.iloc[i,j]):
            continue
        else:
            tv_pred_train.iloc[i,j] = train_raw_ave + user_bias[i] + show_bias[j]
            if tv_pred_train.iloc[i,j] < 1.0:
                tv_pred_train.iloc[i,j] = 1.0
            elif tv_pred_train.iloc[i,j] > 5.0:
                tv_pred_train.iloc[i,j] = 5.0
#tv_pred_train

,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,2.91667,NaN,3.05,2.47727,2.82692
1,2.36667,1.7,2.5,1.92727,2.27692
2,NaN,NaN,4.3,NaN,NaN
3,NaN,NaN,NaN,2.72727,NaN
4,NaN,1.83333,2.63333,NaN,2.41026
5,2.83333,NaN,2.96667,NaN,2.74359
6,2.41667,1.75,2.55,1.97727,NaN
7,NaN,NaN,5,NaN,5
8,NaN,1.16667,1.96667,1.39394,NaN
9,4.16667,NaN,4.3,NaN,NaN


In [209]:
train_bias_rmse = mean_squared_error(tv_train, tv_pred_train.stack().sort_index())**0.5
print("RMSE on the Training Set is: " + str(train_bias_rmse) + "\n")

RMSE on the Training Set is: 1.0347921614021542



In [210]:
tv_pred_test = pd.DataFrame(index=tv_df.index, columns=tv_df.columns.values)
tv_pred_test = tv_pred_test[sorted(tv_pred_test.columns)]
tv_test_wide = tv_test_wide[sorted(tv_test_wide.columns)]
tv_test_wide = tv_test_wide.reindex(tv_df.index)
for l in range(0,len(user_bias)):
    for k in range(0,len(show_bias)):
        if math.isnan(tv_test_wide.iloc[l,k]):
            continue
        else:
            tv_pred_test.iloc[l,k] = test_raw_ave + user_bias[l] + show_bias[k]
            if tv_pred_test.iloc[l,k] < 1.0:
                tv_pred_test.iloc[l,k] = 1.0
            elif tv_pred_test.iloc[l,k] > 5.0:
                tv_pred_test.iloc[l,k] = 5.0
#tv_pred_test

,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,3.77778,NaN,3.91111,NaN,3.68803
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,3.00505,NaN
6,NaN,NaN,NaN,NaN,2.93803
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [211]:
test_bias_rmse = mean_squared_error(tv_test, tv_pred_test.stack().sort_index())**0.5
print("RMSE on the Test Set is: " + str(test_bias_rmse) + "\n")

RMSE on the Test Set is: 1.170815107803852

